https://stackoverflow.com/questions/39604271/conda-environments-not-showing-up-in-jupyter-notebook

Data could be continuous or categorical. There could be missing rows in both the types. For continuous data, we can fill the missing values by taking the mean/median/max value, and create a flag for missing column. eg. salary_missing=1. Once the missing data is added, we can normalize the data. 
For categorical data, they can be converted into pandas data type - category. This will assign a number to each of the categories found in the column. As python treats missing value of category -1, we can add +1 to start the list from 0. We don't need to normalize the data here.

Once this is done, we need to split the data into train, validation and test. 
After this, we need to create an architecture: no of layers, no of nodes in a layer, drop out, activation function, batch normalization.
For judging the model, we can compare the following: training loss, validation loss and accuracy. Comparing training and validation loss can help us in finding cases of underfitting and overfitting.

In [1]:
from fastai import *
from fastai.tabular import * 

In [2]:
path = untar_data(URLs.ADULT_SAMPLE)
path

PosixPath('/home/prateek/.fastai/data/adult_sample')

In [3]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


In [14]:
Normalize??

In [4]:
procs = [FillMissing, Categorify, Normalize]

In [5]:
valid_idx = range(len(df)-2000, len(df))

In [6]:
dep_var = '>=50k'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

In [7]:
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names)  # `cont_names` defaults to: set(df)-set(cat_names)-{dep_var}

['capital-gain', 'age', 'capital-loss', 'hours-per-week', 'education-num', 'fnlwgt']


In [22]:
df.head(1)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1


In [21]:
next(iter(data.train_dl))

[[tensor([[ 5, 10,  3,  5,  1,  5,  2, 40,  1],
          [ 5, 16,  5,  9,  4,  5,  1, 27,  1],
          [ 5, 10,  5, 11,  4,  5,  1, 32,  1],
          [ 5, 16,  5, 11,  3,  3,  1, 40,  1],
          [ 5, 16,  3,  2,  1,  5,  2, 40,  1],
          [ 5, 13,  3,  5,  1,  5,  2, 40,  1],
          [ 5, 12,  3,  7,  1,  5,  2, 40,  1],
          [ 5, 12,  5,  8,  4,  5,  1, 30,  1],
          [ 6, 12,  3,  2,  1,  5,  2, 40,  1],
          [ 5, 16,  7,  2,  5,  5,  1, 40,  1],
          [ 5, 16,  3,  6,  1,  5,  2, 40,  1],
          [ 3, 15,  3,  5,  1,  5,  2, 40,  1],
          [ 5, 12,  3,  5,  1,  5,  2, 40,  1],
          [ 8, 16,  5,  2,  4,  5,  1, 40,  1],
          [ 5, 16,  3,  5,  1,  5,  2, 40,  1],
          [ 3, 12,  3,  5,  1,  5,  2, 40,  1],
          [ 5,  8,  1, 13,  2,  5,  2, 40,  1],
          [ 3, 12,  1, 12,  2,  5,  2, 40,  1],
          [ 5,  1,  3,  4,  1,  5,  2, 40,  1],
          [ 6,  8,  5, 13,  2,  5,  1, 40,  1],
          [ 5, 13,  3, 14,  1,  2,  2, 3

In [8]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[ 7  9  3 11  6  5  1 40  1]
 [ 5  9  1 13  5  5  1 40  1]
 [ 5 10  5  2  4  5  1 40  1]
 [ 5 13  4 11  2  5  2 40  1]
 [ 5 12  5  4  2  3  2 40  1]]
[[-0.145922  0.173985 -0.216787 -2.055581  0.362079 -0.357428]
 [-0.145922 -0.19215  -0.216787 -0.035789  0.362079 -0.52798 ]
 [-0.145922 -1.21733  -0.216787 -0.682123  1.145728 -0.682932]
 [-0.145922  0.759802 -0.216787 -0.035789  1.537552 -0.087228]
 [-0.145922 -0.631513 -0.216787  0.448961 -0.421569  1.478249]]
[1 0 0 0 0]


In [27]:
learn

Learner(data=TabularDataBunch;

Train: LabelList
y: CategoryList (30561 items)
[Category 1, Category 1, Category 0, Category 1, Category 0]...
Path: /home/prateek/.fastai/data/adult_sample
x: TabularList (30561 items)
[TabularLine workclass  Private; education  Assoc-acdm; marital-status  Married-civ-spouse; occupation #na#; relationship  Wife; race  White; sex  Female; native-country  United-States; education-num_na False; capital-gain -0.1459; age 0.7598; capital-loss 4.5067; hours-per-week -0.0358; education-num 0.7539; fnlwgt -0.8371; , TabularLine workclass  Private; education  Masters; marital-status  Divorced; occupation  Exec-managerial; relationship  Not-in-family; race  White; sex  Male; native-country  United-States; education-num_na False; capital-gain 1.2670; age 0.3937; capital-loss -0.2168; hours-per-week 0.3682; education-num 1.5376; fnlwgt 0.4449; , TabularLine workclass  Private; education  HS-grad; marital-status  Divorced; occupation #na#; relationship  Unmarried; r

In [9]:
learn = tabular_learner(data, layers=[200,100], emb_szs={'native-country': 10}, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy
1,0.321034,0.317902,0.853000


In [10]:
learn.predict(df.iloc[0])

(Category 1, tensor(1), tensor([0.2773, 0.7227]))